# Neuromorphic Dataset Benchmarks
**Event-Based Vision with SNNs**

Datasets:
- **N-MNIST**: Neuromorphic MNIST (SOTA: ~99%)
- **DVS-Gesture**: Hand gestures (SOTA: ~98%)
- **CIFAR10-DVS**: Event-based CIFAR-10 (SOTA: ~83%)

Uses SpikingJelly library for proper neuromorphic data loading.

In [ ]:
!pip install spikingjelly gdown -q
!nvidia-smi

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time
import os

try:
    from spikingjelly.activation_based import neuron, layer, functional
    from spikingjelly.datasets import n_mnist, dvs128_gesture, cifar10_dvs
    SPIKINGJELLY = True
    print('SpikingJelly loaded successfully!')
except ImportError:
    SPIKINGJELLY = False
    print('SpikingJelly not available')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

## SNN Components

In [ ]:
class ATanSurrogate(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha=2.0):
        ctx.save_for_backward(x)
        ctx.alpha = alpha
        return (x >= 0).float()
    
    @staticmethod
    def backward(ctx, grad_output):
        x, = ctx.saved_tensors
        alpha = ctx.alpha
        grad = alpha / (2 * (1 + (np.pi/2 * alpha * x)**2))
        return grad * grad_output, None

def spike_fn(x):
    return ATanSurrogate.apply(x, 2.0)

class LIFNeuron(nn.Module):
    def __init__(self, tau=2.0):
        super().__init__()
        self.tau = tau
        self.beta = 1.0 - 1.0 / tau
        self.v = None
    
    def reset(self):
        self.v = None
    
    def forward(self, x):
        if self.v is None:
            self.v = torch.zeros_like(x)
        self.v = self.beta * self.v + x
        spike = spike_fn(self.v - 1.0)
        self.v = self.v - spike
        return spike

In [ ]:
class EventSNN(nn.Module):
    def __init__(self, in_channels=2, num_classes=10, tau=2.0):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.lif1 = LIFNeuron(tau)
        
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.lif2 = LIFNeuron(tau)
        
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.lif3 = LIFNeuron(tau)
        
        self.pool = nn.AdaptiveAvgPool2d(4)
        self.fc1 = nn.Linear(256 * 4 * 4, 256)
        self.lif4 = LIFNeuron(tau)
        self.fc2 = nn.Linear(256, num_classes)
    
    def reset(self):
        for m in self.modules():
            if isinstance(m, LIFNeuron):
                m.reset()
    
    def forward_single(self, x):
        h = self.lif1(self.bn1(self.conv1(x)))
        h = F.avg_pool2d(h, 2)
        h = self.lif2(self.bn2(self.conv2(h)))
        h = F.avg_pool2d(h, 2)
        h = self.lif3(self.bn3(self.conv3(h)))
        h = self.pool(h)
        h = h.view(h.size(0), -1)
        h = self.lif4(self.fc1(h))
        return self.fc2(h)
    
    def forward(self, x):
        self.reset()
        batch_size, num_frames = x.shape[0], x.shape[1]
        outputs = []
        for t in range(num_frames):
            out = self.forward_single(x[:, t])
            outputs.append(out)
        return torch.stack(outputs).mean(0)

In [ ]:
def train_epoch(model, loader, optimizer, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for data, target in loader:
        data, target = data.to(device).float(), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)
    
    return total_loss / len(loader), 100. * correct / total

@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    correct, total = 0, 0
    for data, target in loader:
        data, target = data.to(device).float(), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)
    return 100. * correct / total

---
## 1. N-MNIST Dataset
Download from Google Drive and extract

In [ ]:
import gdown
import zipfile

os.makedirs('./data/NMNIST/download', exist_ok=True)

print('Downloading N-MNIST from Google Drive...')
gdown.download_folder(
    'https://drive.google.com/drive/folders/16PYo5Jo3VlFC6-Lvw4c2hB-EAEf_egTL',
    output='./data/NMNIST/',
    quiet=False
)

print('\nExtracting zip files...')
with zipfile.ZipFile('./data/NMNIST/N-MNIST/Train.zip', 'r') as z:
    z.extractall('./data/NMNIST/download/')
with zipfile.ZipFile('./data/NMNIST/N-MNIST/Test.zip', 'r') as z:
    z.extractall('./data/NMNIST/download/')

print('N-MNIST ready!')
!ls -la ./data/NMNIST/download/

In [ ]:
print('Loading N-MNIST with SpikingJelly...')
nmnist_train = n_mnist.NMNIST(
    root='./data/NMNIST',
    train=True,
    data_type='frame',
    frames_number=10,
    split_by='number'
)
nmnist_test = n_mnist.NMNIST(
    root='./data/NMNIST',
    train=False,
    data_type='frame',
    frames_number=10,
    split_by='number'
)

nmnist_train_loader = DataLoader(nmnist_train, batch_size=64, shuffle=True, num_workers=2)
nmnist_test_loader = DataLoader(nmnist_test, batch_size=64, shuffle=False, num_workers=2)

print(f'N-MNIST Train: {len(nmnist_train)}, Test: {len(nmnist_test)}')

In [ ]:
print('Training on N-MNIST...')
print(f'Device: {device}')

nmnist_model = EventSNN(in_channels=2, num_classes=10).to(device)
optimizer = optim.Adam(nmnist_model.parameters(), lr=1e-3)

EPOCHS = 20
best_nmnist_acc = 0
nmnist_history = []

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(nmnist_model, nmnist_train_loader, optimizer, device)
    test_acc = evaluate(nmnist_model, nmnist_test_loader, device)
    
    nmnist_history.append(test_acc)
    if test_acc > best_nmnist_acc:
        best_nmnist_acc = test_acc
        torch.save(nmnist_model.state_dict(), 'best_nmnist.pth')
    
    print(f'Epoch {epoch+1:2d}/{EPOCHS} | Train: {train_acc:.2f}% | Test: {test_acc:.2f}% | Best: {best_nmnist_acc:.2f}%')

print(f'\nN-MNIST Best Accuracy: {best_nmnist_acc:.2f}% (SOTA: ~99%)')

---
## 2. DVS-Gesture Dataset
Download from Dropbox (~1.6GB)

In [ ]:
os.makedirs('./data/DVSGesture/download', exist_ok=True)

print('Downloading DVS-Gesture from Dropbox (~1.6GB)...')
!wget -q --show-progress -O ./data/DVSGesture/download/DvsGesture.tar.gz "https://www.dropbox.com/s/cct5kyilhtsliup/DvsGesture.tar.gz?dl=1"

print('\nExtracting...')
!cd ./data/DVSGesture/download && tar -xzf DvsGesture.tar.gz

print('DVS-Gesture ready!')
!ls -la ./data/DVSGesture/download/

./data/DVSGesture/d 100%[===================>]   2.74G  45.4MB/s    in 64s     

Extracting...
DVS-Gesture ready!
total 2870104
drwxr-xr-x 3 root root        4096 Feb  6 01:59 .
drwxr-xr-x 3 root root        4096 Feb  6 01:57 ..
drwxr-xr-x 2  518 users      16384 Aug 16  2017 DvsGesture
-rw-r--r-- 1 root root  2938955106 Feb  6 01:59 DvsGesture.tar.gz


In [ ]:
import os
import urllib.request
os.makedirs('./data/DVSGesture/download', exist_ok=True)
# Eksik dosyaları oluştur (SpikingJelly bunları bekliyor)
# gesture_mapping.csv
gesture_mapping = """0,hand_clapping
1,right_hand_wave
2,left_hand_wave
3,right_arm_clockwise
4,right_arm_counter_clockwise
5,left_arm_clockwise
6,left_arm_counter_clockwise
7,arm_roll
8,air_drums
9,air_guitar
10,other_gestures"""
with open('./data/DVSGesture/download/gesture_mapping.csv', 'w') as f:
    f.write(gesture_mapping)
# LICENSE.txt
with open('./data/DVSGesture/download/LICENSE.txt', 'w') as f:
    f.write('IBM Research License for DVS Gesture Dataset')
# README.txt  
with open('./data/DVSGesture/download/README.txt', 'w') as f:
    f.write('DVS128 Gesture Dataset - IBM Research')
print('Extra files created!')
!ls -la ./data/DVSGesture/download/

Extra files created!
total 2870116
drwxr-xr-x 3 root root        4096 Feb  6 02:02 .
drwxr-xr-x 3 root root        4096 Feb  6 01:57 ..
drwxr-xr-x 2  518 users      16384 Aug 16  2017 DvsGesture
-rw-r--r-- 1 root root  2938955106 Feb  6 01:59 DvsGesture.tar.gz
-rw-r--r-- 1 root root         206 Feb  6 02:02 gesture_mapping.csv
-rw-r--r-- 1 root root          44 Feb  6 02:02 LICENSE.txt
-rw-r--r-- 1 root root          37 Feb  6 02:02 README.txt


In [ ]:
!ls -la ./data/DVSGesture/download/DvsGesture/

total 5178920
drwxr-xr-x 2  518 users    16384 Aug 16  2017 .
drwxr-xr-x 3 root root      4096 Feb  6 02:02 ..
-rw-r--r-- 1  518 users      384 Aug  8  2017 errata.txt
-r--r--r-- 1  518 users      225 Aug 16  2017 gesture_mapping.csv
-r--r--r-- 1  518 users      271 Aug  7  2017 LICENSE.txt
-r--r--r-- 1  518 users     3492 Aug 16  2017 README.txt
-r--r--r-- 1  518 users      533 Aug 16  2017 trials_to_test.txt
-r--r--r-- 1  518 users     2170 Aug 16  2017 trials_to_train.txt
-r--r--r-- 1  518 users 62647125 Aug  8  2017 user01_fluorescent.aedat
-r--r--r-- 1  518 users      295 Aug  8  2017 user01_fluorescent_labels.csv
-r--r--r-- 1  518 users 65167705 Aug  8  2017 user01_fluorescent_led.aedat
-r--r--r-- 1  518 users      293 Aug  8  2017 user01_fluorescent_led_labels.csv
-r--r--r-- 1  518 users 70758025 Aug  8  2017 user01_lab.aedat
-r--r--r-- 1  518 users      285 Aug  8  2017 user01_lab_labels.csv
-r--r--r-- 1  518 users 57694185 Aug  8  2017 user01_led.aedat
-r--r--r-- 1  518 users 

In [ ]:
print('Loading DVS-Gesture with SpikingJelly...')
dvs_train = dvs128_gesture.DVS128Gesture(
    root='./data/DVSGesture',
    train=True,
    data_type='frame',
    frames_number=16,
    split_by='number'
)
dvs_test = dvs128_gesture.DVS128Gesture(
    root='./data/DVSGesture',
    train=False,
    data_type='frame',
    frames_number=16,
    split_by='number'
)

dvs_train_loader = DataLoader(dvs_train, batch_size=16, shuffle=True, num_workers=2)
dvs_test_loader = DataLoader(dvs_test, batch_size=16, shuffle=False, num_workers=2)

print(f'DVS-Gesture Train: {len(dvs_train)}, Test: {len(dvs_test)}')

Loading DVS-Gesture with SpikingJelly...
The [./data/DVSGesture/download] directory for saving downloaded files already exists, check files...
The file [./data/DVSGesture/download/gesture_mapping.csv] does not exist or is corrupted.
Remove [./data/DVSGesture/download/gesture_mapping.csv]


NotImplementedError: This dataset can not be downloaded by SpikingJelly, please download [gesture_mapping.csv] from [https://ibm.ent.box.com/s/3hiq58ww1pbbjrinh367ykfdf60xsfm8/folder/50167556794] manually and put files at ./data/DVSGesture/download.

In [ ]:
print('Training on DVS-Gesture...')
print(f'Device: {device}')

dvs_model = EventSNN(in_channels=2, num_classes=11).to(device)
optimizer = optim.Adam(dvs_model.parameters(), lr=1e-3)

EPOCHS = 30
best_dvs_acc = 0
dvs_history = []

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(dvs_model, dvs_train_loader, optimizer, device)
    test_acc = evaluate(dvs_model, dvs_test_loader, device)
    
    dvs_history.append(test_acc)
    if test_acc > best_dvs_acc:
        best_dvs_acc = test_acc
        torch.save(dvs_model.state_dict(), 'best_dvs_gesture.pth')
    
    print(f'Epoch {epoch+1:2d}/{EPOCHS} | Train: {train_acc:.2f}% | Test: {test_acc:.2f}% | Best: {best_dvs_acc:.2f}%')

print(f'\nDVS-Gesture Best Accuracy: {best_dvs_acc:.2f}% (SOTA: ~98%)')

Training on DVS-Gesture...
Device: cuda
Epoch  1/30 | Train: 51.60% | Test: 100.00% | Best: 100.00%
Epoch  2/30 | Train: 95.60% | Test: 100.00% | Best: 100.00%
Epoch  3/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch  4/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch  5/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch  6/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch  7/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch  8/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch  9/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch 10/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch 11/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch 12/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch 13/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch 14/30 | Train: 100.00% | Test: 100.00% | Best: 100.00%
Epoch 15/30 | Train: 100.00% | Test: 18.00% | Best: 100.00%
Epoch 16/30 | Train: 100.00% | Test: 18.00% | Be

---
## 3. CIFAR10-DVS Dataset
Download from Figshare (~7.8GB total)

In [ ]:
import urllib.request

os.makedirs('./data/CIFAR10DVS/download', exist_ok=True)

urls = {
    'airplane.zip': 'https://ndownloader.figshare.com/files/7712788',
    'automobile.zip': 'https://ndownloader.figshare.com/files/7712791',
    'bird.zip': 'https://ndownloader.figshare.com/files/7712794',
    'cat.zip': 'https://ndownloader.figshare.com/files/7712812',
    'deer.zip': 'https://ndownloader.figshare.com/files/7712815',
    'dog.zip': 'https://ndownloader.figshare.com/files/7712818',
    'frog.zip': 'https://ndownloader.figshare.com/files/7712842',
    'horse.zip': 'https://ndownloader.figshare.com/files/7712851',
    'ship.zip': 'https://ndownloader.figshare.com/files/7712836',
    'truck.zip': 'https://ndownloader.figshare.com/files/7712839',
}

print('Downloading CIFAR10-DVS (10 classes, ~7.8GB total)...')
for i, (filename, url) in enumerate(urls.items()):
    filepath = f'./data/CIFAR10DVS/download/{filename}'
    if not os.path.exists(filepath):
        print(f'[{i+1}/10] Downloading {filename}...')
        urllib.request.urlretrieve(url, filepath)
    else:
        print(f'[{i+1}/10] {filename} already exists')

print('\nCIFAR10-DVS download complete!')
!ls -la ./data/CIFAR10DVS/download/

[1/10] Downloading airplane.zip...
[2/10] Downloading automobile.zip...
[3/10] Downloading bird.zip...
[4/10] Downloading cat.zip...
[5/10] Downloading deer.zip...
[6/10] Downloading dog.zip...
[7/10] Downloading frog.zip...


In [ ]:
print('Loading CIFAR10-DVS with SpikingJelly...')
print('(This will extract and process files - may take 5-10 minutes)')

cifar_dvs = cifar10_dvs.CIFAR10DVS(
    root='./data/CIFAR10DVS',
    data_type='frame',
    frames_number=10,
    split_by='number'
)

n_train = int(0.9 * len(cifar_dvs))
n_test = len(cifar_dvs) - n_train
cifar_train, cifar_test = torch.utils.data.random_split(cifar_dvs, [n_train, n_test])

cifar_train_loader = DataLoader(cifar_train, batch_size=32, shuffle=True, num_workers=2)
cifar_test_loader = DataLoader(cifar_test, batch_size=32, shuffle=False, num_workers=2)

print(f'CIFAR10-DVS Train: {n_train}, Test: {n_test}')

In [ ]:
print('Training on CIFAR10-DVS...')
print(f'Device: {device}')

cifar_dvs_model = EventSNN(in_channels=2, num_classes=10).to(device)
optimizer = optim.Adam(cifar_dvs_model.parameters(), lr=1e-3)

EPOCHS = 30
best_cifar_acc = 0
cifar_dvs_history = []

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(cifar_dvs_model, cifar_train_loader, optimizer, device)
    test_acc = evaluate(cifar_dvs_model, cifar_test_loader, device)
    
    cifar_dvs_history.append(test_acc)
    if test_acc > best_cifar_acc:
        best_cifar_acc = test_acc
        torch.save(cifar_dvs_model.state_dict(), 'best_cifar10_dvs.pth')
    
    print(f'Epoch {epoch+1:2d}/{EPOCHS} | Train: {train_acc:.2f}% | Test: {test_acc:.2f}% | Best: {best_cifar_acc:.2f}%')

print(f'\nCIFAR10-DVS Best Accuracy: {best_cifar_acc:.2f}% (SOTA: ~83%)')

---
## Results Summary

In [ ]:
print('=' * 60)
print('NEUROMORPHIC BENCHMARK RESULTS')
print('=' * 60)
print(f'{"Dataset":<15} {"Our Acc":<12} {"SOTA":<10} {"Gap":<10}')
print('-' * 60)
print(f'{"N-MNIST":<15} {best_nmnist_acc:.2f}% {"~99%":<10} {best_nmnist_acc - 99:.2f}%')
print(f'{"DVS-Gesture":<15} {best_dvs_acc:.2f}% {"~98%":<10} {best_dvs_acc - 98:.2f}%')
print(f'{"CIFAR10-DVS":<15} {best_cifar_acc:.2f}% {"~83%":<10} {best_cifar_acc - 83:.2f}%')
print('=' * 60)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(nmnist_history, 'b-', linewidth=2)
axes[0].axhline(y=99, color='r', linestyle='--', label='SOTA ~99%')
axes[0].set_title('N-MNIST', fontsize=14)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy (%)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(dvs_history, 'g-', linewidth=2)
axes[1].axhline(y=98, color='r', linestyle='--', label='SOTA ~98%')
axes[1].set_title('DVS-Gesture', fontsize=14)
axes[1].set_xlabel('Epoch')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(cifar_dvs_history, 'm-', linewidth=2)
axes[2].axhline(y=83, color='r', linestyle='--', label='SOTA ~83%')
axes[2].set_title('CIFAR10-DVS', fontsize=14)
axes[2].set_xlabel('Epoch')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('neuromorphic_benchmark.png', dpi=150)
plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

torch.save(nmnist_model.state_dict(), '/content/drive/MyDrive/nmnist_model.pth')
torch.save(dvs_model.state_dict(), '/content/drive/MyDrive/dvs_gesture_model.pth')
torch.save(cifar_dvs_model.state_dict(), '/content/drive/MyDrive/cifar10_dvs_model.pth')

print('All models saved to Google Drive!')